# Extracting value from the payment notes

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle
import functions as fn
import matplotlib.pyplot as plt 
%matplotlib inline
import emoji
import regex
import nltk
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# load the above mentioned pickle
with open('initial_5pct_transactions.pkl', 'rb') as f:
    initial_5pct = pickle.load(f)

In [ ]:
# Retrieve information to connect to the database
keys = fn.get_keys("/Users/jjherranzsarrion/.secret/local_info.json")
username = keys['username']
password = keys['password']

# Select your time ranges and add them in variables
train_window_end = '2018-07-28 23:59:59'
test_window_start = '2018-07-29 00:00:00'
test_window_end = '2018-07-29 23:59:59'
previous_day_start = '2018-07-28 00:00:00'

In [ ]:
cursor = fn.extracting_cursor(username, password)
q = f"""SELECT *
        FROM payments p;"""
cursor.execute(q)
payments = pd.DataFrame(cursor.fetchall())
payments.columns = [x[0] for x in cursor.description]

In [ ]:
notes = payments['note']

In [ ]:
def split_count(notes):
    """Function that takes in all the notes and returns the emojis used
    in the form of text captured by :colons:"""
    recomposed_note = []
    for note in notes:
        note_text = []
        data = regex.findall(r'\X', note)
        for word in data:
            if any(char in emoji.UNICODE_EMOJI for char in word):
                note_text.append(emoji.demojize(f'{word}'))
            else:
                note_text.append(word)
        recomposed_note.append(''.join(note_text))
    return recomposed_note

In [ ]:
recomposed_note = split_count(notes)

In [ ]:
recomposed_note

In [ ]:
payments['recomposed_note'] = [note for note in split_count(notes)]

In [ ]:
payments

In [ ]:
recomposed_note_freqdist = FreqDist(recomposed_note)
recomposed_note_freqdist.most_common(10)

In [ ]:
stopwords_list = stopwords.words('english')
stopwords_list += ["im", "ur", "u", "'s", "n", "z", "n't"]

recomposed_note_stopped = [note for note in recomposed_note if note not in stopwords_list]

In [ ]:
recomposed_note_stopped

In [ ]:
payments['note'].value_counts()